In [6]:
# !pip install requests
import requests
import pandas as pd

In [7]:
date_params = ["start=1990&end=2000","start=2000&end=2010","start=2010",]
urls = [f"https://api.eia.gov/v2/international/data/?api_key=KB7o3u8fD9dPM6fSnjaG0cBaBBaQ7qyW68Vi52PM&frequency=annual&data[0]=value"
        "&facets[activityId][]=1&facets[productId][]=7&facets[productId][]=130&facets[unit][]=MT&facets[countryRegionTypeId][]=c"
        f"&{d}&sort[0][column]=countryRegionId&sort[0][direction]=asc&offset=0&length=5000" #my API key
        for d in date_params
        ]

In [8]:
dfs = []
for url in urls:
    response = requests.get(url)
    response.raise_for_status()
    df = pd.DataFrame(response.json()["response"]["data"])
    dfs.append(df)

data = pd.concat(dfs)

In [9]:
# Make sure your 'value' column is numeric
data['value'] = pd.to_numeric(data['value'], errors='coerce')

# Pivot the DataFrame
pivot_df = data.pivot_table(
    index=["countryRegionId", "period"],
    columns="productName",
    values="value",
    fill_value=0
).reset_index()

# Rename columns
pivot_df = pivot_df.rename(columns={
    "countryRegionId": "country code",
    "period": "year",
    "Coal": "total coal (MT)",
    "Metallurgical coal": "metallurgical coal (MT)"
})

pivot_df['total coal (MT)'] = pivot_df['total coal (MT)'] / 1000
pivot_df['metallurgical coal (MT)'] = pivot_df['metallurgical coal (MT)'] / 1000

In [10]:
pivot_df.to_csv('eia_production.csv', index=False)